In [113]:
#Max flow problem
from pyomo.environ import *
from pyomo.opt import *
opt = solvers.SolverFactory("glpk")

model = ConcreteModel()
# BIGNUM = 2**40



w = {
    'O': {'A': 5, 'B': 7, 'C': 4},
    'A': {'D': 3, 'B': 1},
    'C': {'E': 4},
    'B': {'D': 4, 'E': 5, 'C': 2},
    'E': {'D':1, 'T': 6},
    'D': {'T': 9}
}
#transform from matrix to pairs so it's easier to work with
W = {(k, i):w[k][i] for k in w for i in w[k] }
# print(W)

A = list(W.keys())

flow_excluded = ['O', 'T']
    

#decision variable: how much we should pass from node i to node j
model.x = Var(A,within=NonNegativeReals)
# maximize the total we can send to the destination 
model.z = Objective(expr=sum( (  model.x[i,j] ) for (i,j) in A if j == 'T'), sense=maximize)
#we can't pass more than the max weight
def flow_rule(model, i, j):
    return model.x[i,j] <= W[i,j]
model.flow = Constraint(A, rule=flow_rule )

def sum_rule(model, n):
    inflow = sum(model.x[i,j] for (i,j) in A if i == n)
    outflow = sum(model.x[i,j] for (i,j) in A if j == n)
    if n not in flow_excluded:
        return inflow == outflow
    else: return Constraint.Skip
model.sum = Constraint(w.keys(), rule=sum_rule)
#maximum amount that can be transported to any given node, k





model.dual = Suffix(direction=Suffix.IMPORT)

results = opt.solve(model)

# assert_optimal_termination(results)
print(model.x.get_values())
print()
print(f"Maximum flow: {model.z.expr()}")


{('O', 'A'): 4.0, ('O', 'B'): 7.0, ('O', 'C'): 3.0, ('A', 'D'): 3.0, ('A', 'B'): 1.0, ('C', 'E'): 3.0, ('B', 'D'): 4.0, ('B', 'E'): 4.0, ('B', 'C'): 0.0, ('E', 'D'): 1.0, ('E', 'T'): 6.0, ('D', 'T'): 8.0}

Maximum flow: 14.0


In [126]:

model.dual.display()


dual : Direction=Suffix.IMPORT, Datatype=Suffix.FLOAT
    Key       : Value
    flow[A,B] :  -0.0
    flow[A,D] :   1.0
    flow[B,C] :   0.0
    flow[B,D] :   1.0
    flow[B,E] :   0.0
    flow[C,E] :   0.0
    flow[D,T] :   0.0
    flow[E,D] :   1.0
    flow[E,T] :   1.0
    flow[O,A] :   0.0
    flow[O,B] :  -0.0
    flow[O,C] :   0.0
       sum[A] :  -0.0
       sum[B] :  -0.0
       sum[C] :  -0.0
       sum[D] :   1.0
       sum[E] :  -0.0


In [ ]:
## the dual variable we are looking for are the ones labeled flow

In [127]:
# (HL, Problem 9.4-3.) The Premiere Bank soon will be hooking up computer terminals at
# each of its branch offices to the computer at its main office using special phone lines with
# telecommunications devices. The phone line from a branch office need not be connected
# directly to the main office. It can be connected indirectly by being connected to another
# branch office that is connected (directly or indirectly) to the main office. The only
# requirement is that every branch office be connected by some route to the main office.
# The charge for the special phone lines is 100 $ times the number of miles involved, where
# the distance (in miles) between every pair of offices is as follows:

SyntaxError: invalid syntax (4027056496.py, line 1)

In [128]:
#(a) Describe how this problem fits the network description of the minimum spanning tree
#problem
'''This problem fits the description of MST since we branch offices can be connected indirectly by being connected to the other office, this means that the optimal structure we want to build is acyclical, it also means that we only need to connect all vertices in our graph with the minimum possible sum of edge weights'''

In [ ]:
model2 = ConcreteModel()
A = ['M','B1', 'B2', 'B3', 'B4', 'B5']
E = {('M', 'B1'): 190, ('M', 'B2'): 70, ('M', 'B3'): 115, ('M', 'B4'): 270, ('M', 'B5'): 160,
     ('B1', 'B2'): 100, ('B1', 'B3'): 110, ('B1', 'B4'): 215, ('B1', 'B5'): 50,
     ('B2', 'B3'): 140, ('B2', 'B4'): 120, ('B2', 'B5'):220,
     ('B3', 'B4'): 175, ('B3', 'B5'): 80, 
     ('B4', 'B5'): 310 }
for (i,j) in E:
    E[j,i] = E[i,j]


model.x = Var(A,within=NonNegativeReals)
# maximize the total we can send to the destination 
model.z = Objective(expr=sum( (  model.x[i,j]*E[i,j] ) for (i,j) in E), sense=minimize)
# spanning tree can only have n-1 edges
model.min_edges = Constraint(expr= sum(x[i,j] for (i,j) in E) == len(A) -1 ) 
